In [1]:
import pandas as pd
import sqlite3

# Create connection to HR database
conn = sqlite3.connect('assets/data/hr_database.sqlite')

# Create cursor object
cursor = conn.cursor()

# Get list of tables
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]

# Read data from each table into dataframes
dfs = {}
for table in table_names:
    dfs[table] = pd.read_sql_query(f"SELECT * FROM {table}", conn)

# Close connection
conn.close()


In [24]:
import json

def fix_column_quotes(config_path):
    # Reahe current configuration
   with open(config_path, 'r') as f:
       config = json.load(f)
   
   # Fix each rule's python_code
   for rule_key, rule in config['rule_configs'].items():
       python_code = rule['python_code']
       column_name = rule['column_name']
       
       # Replace single quoted or unquoted column references with triple quotes
       python_code = python_code.replace(f"df['{column_name}']", f"df['''{column_name}''']")
       python_code = python_code.replace(f'df["{column_name}"]', f"df['''{column_name}''']")
       python_code = python_code.replace(f"df[{column_name}]", f"df['''{column_name}''']")
       
       # Update the rule
       config['rule_configs'][rule_key]['python_code'] = python_code
   
   # Write the updated configuration
   with open(config_path, 'w') as f:
       json.dump(config, f, indent=2)
# Usage
fix_column_quotes('rule_configurations.json')

In [25]:
len(dfs), list(dfs.keys())

(12,
 ['person_assignments',
  'sqlite_sequence',
  'person_addresses',
  'per',
  'assignments',
  'addresses',
  'rule_validation_results',
  'validation_results',
  'rule_configurations',
  'rule_execution_status',
  'rule_execution_control',
  'rule_results_cache'])

In [26]:
dfs['rule_configurations']

,id,rule_id,table_name,column_name,active,created_date,last_modified,configuration_json
0,1,dq1,addresses,street_address,1,2024-12-20 18:04:32,2024-12-20 18:12:27,{}
1,2,dq1,person_addresses,per_id,1,2024-12-20 18:06:10,2024-12-20 18:06:10,{}
2,3,dq1,addresses,country,1,2024-12-20 18:07:53,2024-12-20 18:07:53,{}
3,5,gdpr_001,per,name,1,2024-12-20 18:14:15,2024-12-20 18:14:15,{}


In [15]:


# Check for Social Security Numbers
# dfs['per']['nino'].str.match(r'\b\d{3}-\d{2}-\d{4}\b')

# Check for valid age range
# dfs['per']['age'].between(18, 100)

# Check for valid email format
# dfs['per']['company'].str.match(r'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b')


0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: nino, Length: 1000, dtype: bool

In [16]:
# dfs['per']['age'].str.match(r'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b') #  data types matters.

In [23]:
dfs['per']['''name'''].str.match(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Name: name, Length: 1000, dtype: bool